# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing Turi Create functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [1]:
from __future__ import division
import turicreate as tc
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = tc.SFrame('amazon_baby.sframe/')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

**Aparte**. En este cuaderno, eliminamos todos los signos de puntuación en aras de la simplicidad. Un enfoque más inteligente de las puntuaciones preservaría frases como "I'd", "would've", "haven't", etc. Consulte  [esta pagina](https://www.cis.upenn.edu/~treebank/tokenization.html) para ver un ejemplo de manejo inteligente de puntuaciones.

In [5]:
import string 
def remove_punctuation(text):
    try: # python 2.x
        text = text.translate(None, string.punctuation) 
    except: # python 3.x
        translator = text.maketrans('', '', string.punctuation)
        text = text.translate(translator)
    return text

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = tc.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1.0,'stripping': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'this':1.0, 'for': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0,'headachesthanks': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1.0, 'for': 1.0,'barnes': 1.0, 'at': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ..."


In [7]:
products[269]['word_count']

{'our': 1.0, 'in': 1.0, 'favorite': 1.0, 'house': 1.0, 'a': 1.0}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1.0,'disappointed': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1.0, 'this':1.0, 'for': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1.0, 'clever':1.0, 'binky': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1.0,'headachesthanks': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1.0, 'for': 1.0,'barnes': 1.0, 'at': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1.0, 'now':1.0, 'postits': 1.0, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [10]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [11]:
sentiment_model = tc.logistic_classifier.create(train_data,
                                                        target = 'sentiment',
                                                        features=['word_count'],
                                                        validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.778177  | 2.742215     | 0.942638          |

| 3         | 12       | 2.778177  | 3.040352     | 0.967822          |

| 4         | 13       | 2.778177  | 3.296576     | 0.976495          |

| 5         | 14       | 2.778177  | 3.559408     | 0.976495          |

+-----------+----------+-----------+--------------+-------------------+

In [12]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 5
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 1.9787

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[offsi]              : 21.7657
word_count[feedthrough]        : 21.6818
word_count[conclusions]        : 21.6818
word_count[easycheap]          : 21.6818
word_count[torsional]          : 21.6818

Lowest Negative Coefficients
----------------------------
word_count[wahwah]

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

**Aparte**. Es posible que reciba una advertencia del tipo "Terminado debido a dificultades numéricas --- este modelo puede no ser el ideal". Significa que la métrica de calidad (que se tratará en el Módulo 3) no mejoró en la última iteración de la ejecución. La dificultad surge cuando el modelo de sentimiento pone demasiado peso en palabras extremadamente raras. Una forma de rectificar esto es aplicar la regularización, que se tratará en el Módulo 4. La regularización disminuye el efecto de las palabras extremadamente raras. Sin embargo, a los efectos de esta tarea, proceda con el modelo anterior.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [14]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [15]:
weights

name,index,class,value,stderr
(intercept),None,1,0.7797362814628332,None
word_count,recommend,1,0.305245773758288,None
word_count,disappointed,1,-2.426792494478245,None
word_count,wise,1,-0.18170817410611284,None
word_count,love,1,0.8405057320615046,None
word_count,it,1,0.010479446044200423,None
word_count,planet,1,-0.3846729911238384,None
word_count,and,1,0.03631437278658715,None
word_count,bags,1,0.1305759809887294,None
word_count,wipes,1,-0.0020751803190896334,None


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [19]:
num_positive_weights = (weights['value'] >=0 ).sum()
num_negative_weights = (weights['value'] < 0 ).sum()

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


**Quiz Question:** How many weights are >= 0?

**R/** 91073

OTRA FORMA

In [20]:
num_positive_weights = len(weights[weights['value']>=0])
num_negative_weights = len(weights[weights['value']<0])

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 91073 
Number of negative weights: 30640 


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [21]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1.0, 'book':1.0, 'same': 1.0, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1.0, 'waste': 1.0,'wouldnt': 1.0, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [22]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [23]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [30]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')#forma de extraer la suma de los WTh(x), con 'margin'
print(scores)

[4.788907309214061, -3.0007822224626497, -8.188501360762766]


In [71]:
len(scores)

3

### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [68]:
def class_prediction (scores):
    prediction = {}  
    for i in range(len(scores)):
    
        if scores[i] > 0:
            prediction[i] = +1.0
        else:
            prediction[i] = -1.0
    
    return (prediction)

In [69]:
class_prediction(scores)

{0: 1.0, 1: -1.0, 2: -1.0}

In [65]:
yhat = [1 if score >0 else -1 for score in scores]
yhat

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [73]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to Turi Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from Turi Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [83]:
from math import exp, expm1

def calculate_probability (scores):
    probability_predictions ={}
    for i in range(len(scores)):
        probability_predictions[i] = 1 / (1 + exp(-scores[i]))
        
    return (probability_predictions)

In [87]:
calculate_probability(scores)

{0: 0.991747131328689, 1: 0.047390547487115554, 2: 0.00027775277121726174}

In [95]:
help(sentiment_model.predict)

Help on method predict in module turicreate.toolkits.classifier.logistic_classifier:

predict(dataset, output_type='class', missing_value_action='auto') method of turicreate.toolkits.classifier.logistic_classifier.LogisticClassifier instance
    Return predictions for ``dataset``, using the trained logistic
    regression model. Predictions can be generated as class labels,
    probabilities that the target value is True, or margins (i.e. the
    distance of the observations from the hyperplane separating the
    classes). `probability_vector` returns a vector of probabilities by
    each class.
    
    For each new example in ``dataset``, the margin---also known as the
    linear predictor---is the inner product of the example and the model
    coefficients. The probability is obtained by passing the margin through
    the logistic function. Predicted classes are obtained by thresholding
    the predicted probabilities at 0.5. If you would like to threshold
    predictions at a diffe

In [96]:
dir(sentiment_model.predict)#podemos ver rápidamente que métodos podemos usar para la variable que estamos usando.


['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [94]:
print("Class predictions according to Turi Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to Turi Create:
[0.9917471313286889, 0.04739054748711556, 0.0002777527712172618]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

**R/** Third data point in *sample_test_data*

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [107]:
test_data['probability'] = sentiment_model.predict(test_data, output_type='probability')


In [123]:
most_positive_reviews = test_data['name','probability'].topk('probability', k=20)

In [126]:
print(most_positive_reviews.print_rows(num_rows=20))

+-------------------------------+-------------+
|              name             | probability |
+-------------------------------+-------------+
| Fisher-Price Cradle 'N Swi... |     1.0     |
| The Original CJ's BuTTer (... |     1.0     |
| Baby Jogger City Mini GT D... |     1.0     |
| Diono RadianRXT Convertibl... |     1.0     |
| Diono RadianRXT Convertibl... |     1.0     |
| Graco Pack 'n Play Element... |     1.0     |
| Maxi-Cosi Pria 70 with Tin... |     1.0     |
| Britax 2012 B-Agile Stroll... |     1.0     |
| Quinny 2012 Buzz Stroller,... |     1.0     |
| Roan Rocco Classic Pram St... |     1.0     |
| Britax Decathlon Convertib... |     1.0     |
| bumGenius One-Size Snap Cl... |     1.0     |
| Infantino Wrap and Tie Bab... |     1.0     |
| Baby Einstein Around The W... |     1.0     |
| Britax Frontier Booster Ca... |     1.0     |
| Evenflo X Sport Plus Conve... |     1.0     |
| P'Kolino Silly Soft Seatin... |     1.0     |
| Peg Perego Aria Light Weig... |     1.

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [128]:
less_positive_reviews = test_data['name','probability'].topk('probability', k=20, reverse=True)
print(less_positive_reviews.print_rows(num_rows=20))

+-------------------------------+------------------------+
|              name             |      probability       |
+-------------------------------+------------------------+
| Luna Lullaby Bosom Baby Nu... | 3.2297908424316137e-63 |
| The First Years True Choic... | 1.632282318647586e-24  |
| Jolly Jumper Arctic Sneak ... | 8.110311382150216e-20  |
| Motorola MBP36 Remote Wire... | 7.797281605719576e-16  |
| VTech Communications Safe ... | 1.841161479868227e-14  |
| Fisher-Price Ocean Wonders... |  6.34509494188516e-14  |
| Levana Safe N'See Digital ... | 6.578528513086744e-14  |
| Safety 1st High-Def Digita... | 1.5408410011909647e-13 |
| Snuza Portable Baby Moveme... | 6.301835289201094e-13  |
| Adiri BPA Free Natural Nur... | 9.314560905770575e-13  |
| Samsung SEW-3037W Wireless... | 5.920602970156652e-12  |
| Motorola Digital Video Bab... | 5.986200850023912e-12  |
| Cloth Diaper Sprayer--styl... | 1.0091402072231447e-11 |
| Munchkin Nursery Projector... | 3.4551832745662225e-11

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [129]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correctly_classified = (predictions==true_labels).sum()

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correctly_classified / len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [130]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**R/** = 0.92

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

**R/** NO

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [131]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [132]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

Para cada revisión, usaremos la columna word_count y eliminaremos todas las palabras que no estén en la lista de palabras significativas anterior

In [133]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [135]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [136]:
print(train_data[0]['word_count'])

{'recommend': 1.0, 'disappointed': 1.0, 'wise': 1.0, 'love': 1.0, 'it': 3.0, 'planet': 1.0, 'and': 3.0, 'bags': 1.0, 'wipes': 1.0, 'highly': 1.0, 'not': 2.0, 'early': 1.0, 'came': 1.0, 'i': 1.0, 'does': 1.0, 'my': 2.0, 'was': 1.0, 'now': 1.0, 'wipe': 1.0, 'holder': 1.0, 'leak': 1.0, 'keps': 1.0, 'osocozy': 1.0, 'moist': 1.0}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [137]:
print(train_data[0]['word_count_subset'])

{'disappointed': 1.0, 'love': 1.0}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [138]:
simple_model = tc.logistic_classifier.create(train_data,
                                                     target = 'sentiment',
                                                     features=['word_count_subset'],
                                                     validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.335615     | 0.866815          |

| 6         | 7        | 0.394815     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.405

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wor

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [139]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [140]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552027043,0.012088854133053259
word_count_subset,disappointed,1,-2.3550925006107253,0.050414988855697916
word_count_subset,love,1,1.3654354936790372,0.03035462951090517
word_count_subset,well,1,0.5042567463979284,0.021381300630990033
word_count_subset,product,1,-0.320555492995575,0.015431132136201635
word_count_subset,loves,1,1.6772714555592918,0.04823282753835012
word_count_subset,little,1,0.5206286360250184,0.021469147566490373
word_count_subset,work,1,-0.6217000124253143,0.023033059794584827
word_count_subset,easy,1,1.1936618983284648,0.029288869202029586
word_count_subset,great,1,0.9446912694798443,0.020950992659050018


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [141]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |  1.6772714555592918  |
| word_count_subset |   perfect    |   1   |  1.5144862670271348  |
| word_count_subset |     love     |   1   |  1.3654354936790372  |
|    (intercept)    |     None     |   1   |  1.2995449552027043  |
| word_count_subset |     easy     |   1   |  1.1936618983284648  |
| word_count_subset |    great     |   1   |  0.9446912694798443  |
| word_count_subset |    little    |   1   |  0.5206286360250184  |
| word_count_subset |     well     |   1   |  0.5042567463979284  |
| word_count_subset |     able     |   1   |  0.1914383022947509  |
| word_count_subset |     old      |   1   |  0.0853961886678159  |
| word_count_subset |     car      |   1   | 0.05883499006802042  |
| word_count_subset |     less     |   1   | -0.

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [143]:
weights_simple_model = simple_model.coefficients

num_positive_weights_simple_model = (weights_simple_model['value'] >=0 ).sum()
num_negative_weights_simple_model = (weights_simple_model['value'] < 0 ).sum()

print("Number of positive weights: %s " % num_positive_weights_simple_model)
print("Number of negative weights: %s " % num_negative_weights_simple_model)

Number of positive weights: 11 
Number of negative weights: 10 


**R/** = 10

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

**R/** YES

In [156]:
simple_weights = simple_model.coefficients
simple_weights

name,index,class,value,stderr
(intercept),None,1,1.2995449552027043,0.012088854133053259
word_count_subset,disappointed,1,-2.3550925006107253,0.050414988855697916
word_count_subset,love,1,1.3654354936790372,0.03035462951090517
word_count_subset,well,1,0.5042567463979284,0.021381300630990033
word_count_subset,product,1,-0.320555492995575,0.015431132136201635
word_count_subset,loves,1,1.6772714555592918,0.04823282753835012
word_count_subset,little,1,0.5206286360250184,0.021469147566490373
word_count_subset,work,1,-0.6217000124253143,0.023033059794584827
word_count_subset,easy,1,1.1936618983284648,0.029288869202029586
word_count_subset,great,1,0.9446912694798443,0.020950992659050018


In [158]:
#para quitar el intercepto de hace el & entre value y name
positive_significant_words = simple_weights[(simple_weights['value'] > 0) & (simple_weights['name'] == "word_count_subset")]['index']
print (len(positive_significant_words))
print (positive_significant_words)

10
['love', 'well', 'loves', 'little', 'easy', 'great', 'able', 'perfect', 'old', 'car']


In [164]:
weights = sentiment_model.coefficients
weights

name,index,class,value,stderr
(intercept),None,1,0.7797362814628332,None
word_count,recommend,1,0.305245773758288,None
word_count,disappointed,1,-2.426792494478245,None
word_count,wise,1,-0.18170817410611284,None
word_count,love,1,0.8405057320615046,None
word_count,it,1,0.010479446044200423,None
word_count,planet,1,-0.3846729911238384,None
word_count,and,1,0.03631437278658715,None
word_count,bags,1,0.1305759809887294,None
word_count,wipes,1,-0.0020751803190896334,None


In [165]:
weights.filter_by(positive_significant_words, 'index')

name,index,class,value,stderr
word_count,love,1,0.8405057320615046,None
word_count,well,1,0.40107557492331714,None
word_count,loves,1,0.9749823125142639,None
word_count,little,1,0.4099316272571677,None
word_count,easy,1,0.734982625567491,None
word_count,great,1,0.7789532883805074,None
word_count,able,1,0.10752802191423969,None
word_count,perfect,1,1.044799420404867,None
word_count,old,1,0.07967490900987581,None
word_count,car,1,0.1196578765076586,None


In [162]:
#para quitar el intercepto de hace el & entre value y name
all_positive_significant_words = weights[(weights['value'] > 0) & (weights['name'] == "word_count")]['index']
print (len(all_positive_significant_words))
print (all_positive_significant_words)

91072
['recommend', 'love', 'it', 'and', 'bags', 'highly', 'early', 'came', 'i', 'my', 'now', 'wipe', 'holder', 'keps', 'osocozy', 'moist', 'quilt', 'this', 'for', 'anyone', 'to', 'perfectlywould', 'bed', 'looking', 'soft', 'type', 'very', 'looksfit', 'size', 'comfortable', 'full', 'than', 'tool', 'clever', 'binky', 'approach', 'getting', 'like', 'is', 'herself', 'has', 'well', 'so', 'fairy', 'worth', 'found', 'ingenious', 'most', 'have', 'what', 'her', 'product', 'about', 'how', 'much', 'purchase', 'artwork', 'daughter', 'in', 'she', 'a', 'proud', 'loves', 'little', 'headachesthanks', 'many', 'will', 'thumbuddy', 'pacifier', 'their', 'them', 'ween', 'you', 'when', 'kids', 'gift', 'nonstop', 'with', 'an', 'book', 'easy', 'parents', 'help', 'from', 'must', 'itthis', 'great', 'or', 'important', 'lots', 'sucking', 'parent', 'house', 'special', 'any', 'prepare', 'our', 'cute', 'habit', 'comes', ... ]


In [163]:
import numpy as np

positive_significant_words = np.array(positive_significant_words)
all_positive_significant_words = np.array(all_positive_significant_words)

np.intersect1d(positive_significant_words,all_positive_significant_words)

array(['able', 'car', 'easy', 'great', 'little', 'love', 'loves', 'old',
       'perfect', 'well'], dtype='<U215')

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [144]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])


0.976494573364514

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [145]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

**R/** = SENTIMENT_MODEL

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [146]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9221862251019919

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [147]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

**R/** = SENTIMENT_MODEL

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

Es bastante común usar el **clasificador de clase mayoritaria** como modelo de referencia (o referencia) para compararlo con su modelo de clasificador. El modelo de clasificador mayoritario predice la clase mayoritaria para todos los puntos de datos. Como mínimo, debería vencer al clasificador de clase mayoritario de manera saludable; de lo contrario, el modelo (generalmente) no tiene sentido.

What is the majority class in the **train_data**?

**R/** = MAJORITY CLASS IS +1 (NUMBER POSITIVE)

In [152]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


In [154]:
accuracy_major_train_data = num_positive / (num_positive + num_negative)
accuracy_major_train_data

0.8407087605684476

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [149]:
num_positive_test_data = (test_data['sentiment'] == +1).sum()
num_negative_test_data = (test_data['sentiment'] == -1).sum()
print(num_positive_test_data)
print(num_negative_test_data)

28095
5241


In [150]:
accuracy_major_test_data = num_positive_test_data / (num_positive_test_data + num_negative_test_data)
accuracy_major_test_data

0.8427825773938085

OTRA FORMA

In [151]:
accuracy_major_test_data = num_positive_test_data / len(test_data['sentiment'])
accuracy_major_test_data

0.8427825773938085

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

**R/** = YES